In [ ]:
#>>> I = numpy.asarray(PIL.Image.open('test.jpg'))

#>>> im = PIL.Image.fromarray(numpy.uint8(I))

In [39]:
from PIL import Image 
import numpy
import torch

xs = []
ys = []

for i in range(0, 10):
    img = Image.open(f'images/images/image_{i}.jpeg')
    I = numpy.asarray(img)
    #print("data", I)
    
    xs.append([257,257,257,0,0])
    ys.append(I[0][0].tolist())


    for rows in range(0, 16):
        for cols in range(0,16):
            if rows==15 and cols ==15:
                continue

            temp_x = I[rows][cols].tolist()
            temp_x.append(rows)
            temp_x.append(cols)

            xs.append(temp_x)
            if cols == 15:
                ys.append(I[rows+1][0].tolist())
            else:
                ys.append(I[rows][cols+1].tolist())
            
xs = torch.tensor(xs)
ys = torch.tensor(ys)

#print("Xs", xs)
#print("Ys", ys)
